In [ ]:
import copy
import numpy as np
import random
import time
from sct_utils import *
import matplotlib.pyplot as plt
from numba import jit

In [ ]:
### GLOBAL CONSTANTS
T = 10; J = 11; Q = 12; K = 13; A = 14
empt = np.array([0,0]) # represents a card absent from the board
s = np.zeros([15,2],int); s[:,0]=np.arange(15); s[:,1]=1 #spades
h = np.copy(s); h[:,1]+=1 # hearts
d = np.copy(s); d[:,1]+=2 # diamonds
c = np.copy(s); c[:,1]+=3 # clubs
values = {2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9', 10:'10', 11:'J', 12:'Q', 13:'K', 14:'A'}
suits = {1:'s', 2:'h', 3:'d', 4:'c'}

In [ ]:
def random_hand_generator(n_p = 2):
    
    deck = np.random.permutation([list(card) for card in np.concatenate((s[2:],h[2:],d[2:],c[2:]))])
    p = np.zeros([6,6,2],int)
    
    for i in range(n_p):
        p[i] = deck[6*i:6*i+6]
        
    return p

In [ ]:
# triplecon continues generating new boards until the players'
# estimated equities have not changed much in the last few iterations
# which is a proxy for convergence

def triplecon(p,b,con_num,con_int):
    start_time=time.time()
    deck=[list(card) for card in np.concatenate((s[2:],h[2:],d[2:],c[2:]))]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~ Encode number of players
    n_p=int(np.sum(p>0)/12)
    print('number of players = ',n_p)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~ Encode number of cards on each board & round-number
    nc=int(np.sum(b>0)/6)
    if nc==0:
        rd='preflop'
    elif nc==3:
        rd='flop'
    elif nc==4:
        rd='turn'
    elif nc==5:
        rd='river'
    print('number of cards on each board = ',nc)
    print('round = ',rd)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~ Print  players & boards
    for i in range(n_p):
        print("Player ", i, "'s cards': ", end=' ')
        for j in range(6):
            print(values[p[i,j,0]]+suits[p[i,j,1]], end=' ')
        print()
    print()

    for i in range(3):
        print("Board ", i, ": ", end=' ')
        for j in range(nc):
            print(values[b[i,j,0]]+suits[b[i,j,1]], end=' ')
        print()
    print()
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~ Remove cards from deck
    for i in range(n_p):
        for j in range(6):
            deck.remove(list(p[i,j]))
    for i in range(3):
        for j in range(nc):
            deck.remove(list(b[i,j]))
    #print(deck)
    decksave=copy.copy(deck);
    #print("DECKSAVE",decksave)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~ Generate a random completed board and modify running totals
    pe=np.zeros([n_p,1])
    fin_it=np.zeros([1])
    print('preamble_length=',time.time()-start_time)
    chstr_loop_total=0
    start_loop_time=time.time()
    if rd!='river':
        itno=0
        maxdiff=np.array([]) # maximum change in any player's estimated equity in the past con_num iterations
        
        fin_it[0]=time.time()
        while True:
            print("-------------- NEW ITERATION --------------")
#             print('LENGTH', len(deck))
#             print('deck', deck)
            itno+=1
            for i in range(3): # generates a random completion of the boards
                for j in range(nc,5):
                    b[i,j]=deck[random.randrange(0,len(deck))]
                    deck.remove(list(b[i,j]))
                    
            # ph is the players' hand strength in vector form
            ph=np.zeros([3,n_p,6],int)
            for i in range(3):
#                 print("Board ", i, ": ", end=' ')
#                 for j in range(5):
#                     print(values[b[i,j,0]]+suits[b[i,j,1]], end=' ')
#                 print()
                for j in range(n_p):
                    chstr_loop_begin=time.time()
                    ph[i,j,:]=hand(p[j],b[i])
                    chstr_loop_total+=time.time()-chstr_loop_begin
#                     print("P ", j, ": ", ph[i,j])

            # pn represents ph numerically in base-15 2nd INDEX ARE BOARDS
            pn=np.sum(ph*[15**n for n in [5,4,3,2,1,0]],2).transpose()

            spn=np.sort(pn,0)[::-1]; ipn=np.argsort(pn,0)[::-1] #sorted pn and indices

            wnrs=sum(spn[0,:]==spn); #number of winners on each board

            pb=np.zeros([n_p,3])
            for i in range(3):
                pb[ipn[0:wnrs[i],i],i]=1/wnrs[i] #pb[i,j] is pts won by Pi on Bj

            pt=np.sum(pb,1)

            spt=np.sort(pt,0)[::-1]; ipt=np.argsort(pt,0)[::-1] #sorted pn and indices

            wnrst=sum(spt[0]==spt)

            pbt=np.zeros([n_p])
            pbt[ipt[0:wnrst]]=1/wnrst

            pe=np.append(pe,[ [ ((itno-1)*pe[k,itno-1] + pbt[k])/itno ] for k in range(n_p) ],1)
            print("Iteration ",itno,", pe[:,",itno,"] = ",pe[:,itno])
            deck=copy.copy(decksave);

            fin_it=np.append(fin_it,time.time())

            if itno>con_num: #this if statement checks (once per con_num iterations) if the last con_num (triplet)
                             #values of playerequities fall within a neighbourhood of size con_it
                             # ie. checks if any of the player's estimated equities have changed more than con_it in the last
                             # con_num iterations
                maxdiff=np.append(maxdiff,max([abs(pe[k,n]-pe[k,itno]) for n in range(itno-con_num,itno) for k in range(n_p)]))
                if (itno/con_num).is_integer():
                    print("maxdiff ",itno," = ",maxdiff[-1],)
                if maxdiff[-1]<con_int:
                    break
    postamble_start=time.time()                
    fin_it=1000*(fin_it-start_loop_time)
    #print("fin_it = ",fin_it)

    diff_fin_it=np.diff(fin_it)
    #print("diff_fin_it = ",diff_fin_it)
    print("number of iterations = ",itno)
    average_iteration_time=np.mean(diff_fin_it)
    print("avg iteration time = ",round(average_iteration_time,2),'ms')
    print("its per second = ",round(1000/average_iteration_time,2))

    #print("pe = \n",np.array([[round(pe[k,z],4) for z in range(itno+1)] for k in range(n_p)]))
    pe=np.append(pe,np.zeros([6-n_p,pe.shape[1]]),0)

    plt.figure(figsize=(12, 6))
    plt.grid(True)
    plt.title("Players' estimated equity at each iteration (r,b,g,y,o,k)")
    plt.plot(range(1,itno+1),pe[0,1:],'r',range(1,itno+1),pe[1,1:],'b',range(1,itno+1),pe[2,1:],'g',range(1,itno+1),pe[3,1:],'y',range(1,itno+1),pe[4,1:],'o',range(1,itno+1),pe[5,1:],'k')

    plt.figure(figsize=(12,3))
    plt.plot(range(con_num+1,len(pe[0,:])),maxdiff)

    plt.figure(figsize=(12,2))
    plt.plot(range(2*con_num+1,len(pe[0,:])),maxdiff[con_num:itno-1])

    print("Estimated equities")
    for i in range(n_p):
        print("Player ",i,": ",pe[i,-1])

    #print(tpiver(np.array([[4,2], [7,1], [14,2], [5,1], [6,2]])))
    print("Total elapsed time = ",time.time()-start_time," seconds")
    print('postamble length = ',time.time()-postamble_start)
    print('chstr loop total = ',chstr_loop_total)

In [ ]:
# #~~~~~Enter player hands and iterations
# Either enter manually:
# p=np.zeros([6,6,2],int)
# p[0]=np.array([ c[A] , s[Q] , c[K] , d[J] , d[T] , s[A] ])
# p[1]=np.array([ h[2] , c[2] , s[2] , d[2] , c[7] , c[Q] ])
# p[2]=np.array([ empt , empt , empt , empt , empt , empt ])
# p[3]=np.array([ empt , empt , empt , empt , empt , empt ])
# p[4]=np.array([ empt , empt , empt , empt , empt , empt ])
# p[5]=np.array([ empt , empt , empt , empt , empt , empt ])

# or generate randomly:
n_p = 3
pc = random_hand_generator(n_p) # players cards
for i in range(n_p):
    print("Player ", i, ": ", end=' ')
    for j in range(6):
        print(values[pc[i,j,0]]+suits[pc[i,j,1]], end=' ')
    print()
print()

boards=np.zeros([3,5,2],int)

# Enter boards manually:
#b[0]=np.array([ c[Q] , s[6] , c[4] , d[Q] , empt ])
#b[1]=np.array([ c[2] , s[2] , c[5] , h[Q] , empt ])
#b[2]=np.array([ h[5] , c[J] , d[8] , s[7] , empt ])

# Or leave empty for a preflop start
boards[0]=np.array([empt, empt, empt, empt, empt])
boards[1]=np.array([empt, empt, empt, empt, empt])
boards[2]=np.array([empt, empt, empt, empt, empt])

In [ ]:
# Run the above block first to enter hands and boards (or generate randomly) before running this
triplecon(pc,boards,40,0.0001)